In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from email.message import EmailMessage
import smtplib
import ssl
from pathlib import Path
from datetime import datetime

In [10]:

conn = psycopg2.connect("dbname=kblc user=postgres password=1234 host=localhost")

query_power_usage = """
SELECT substation_id, timestamp, usage_kwh
FROM kblc_data.power_usage
WHERE timestamp >= NOW() - INTERVAL '1 day';
"""
df_power_usage = pd.read_sql(query_power_usage, conn)

query_outages = """
SELECT outage_id, area_id, start_time, end_time, duration_minutes
FROM kblc_data.outage_logs
WHERE start_time >= NOW() - INTERVAL '7 days';
"""
df_outages = pd.read_sql(query_outages, conn)

query_transformer = """
SELECT transformer_id, timestamp, temperature_c, vibration_level, status
FROM kblc_data.transformer_health
WHERE timestamp >= NOW() - INTERVAL '1 day';
"""
df_transformer = pd.read_sql(query_transformer, conn)

query_billing = """
SELECT customer_id, billing_cycle_start, billing_cycle_end, actual_kwh, deviation_percent, anomaly_type
FROM kblc_data.billing_anomalies
WHERE billing_cycle_end >= NOW() - INTERVAL '1 month';
"""
df_billing = pd.read_sql(query_billing, conn)


/tmp/ipykernel_578071/3318157884.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_power_usage = pd.read_sql(query_power_usage, conn)
/tmp/ipykernel_578071/3318157884.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_outages = pd.read_sql(query_outages, conn)
/tmp/ipykernel_578071/3318157884.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_transformer = pd.read_sql(query_transformer, conn)
/tmp/ipykernel_578071/3318157884.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) o

In [12]:
# --- Step 1: Data Analysis ---

# Daily power usage summary
daily_usage = df_power_usage.groupby('substation_id').usage_kwh.sum().reset_index()
threshold = daily_usage.usage_kwh.mean() + 2 * daily_usage.usage_kwh.std()
daily_usage['anomaly'] = daily_usage.usage_kwh > threshold
anomaly_count = daily_usage['anomaly'].sum()
total_substations = daily_usage.shape[0]
total_consumption = daily_usage['usage_kwh'].sum()

# Outage summary
df_outages['duration_minutes'] = (pd.to_datetime(df_outages['end_time']) - pd.to_datetime(df_outages['start_time'])).dt.total_seconds() / 60
outage_summary = df_outages.groupby('area_id').agg({
    'outage_id': 'count',
    'duration_minutes': 'mean'
}).rename(columns={'outage_id': 'outage_count', 'duration_minutes': 'avg_duration'})
total_areas = outage_summary.shape[0]
avg_outage_duration = outage_summary['avg_duration'].mean()
max_outage_area = outage_summary['outage_count'].idxmax()
max_outage_freq = outage_summary.loc[max_outage_area, 'outage_count']

# Transformer alerts
df_transformer['temp_alert'] = df_transformer.temperature_c > 80
total_transformers = df_transformer.shape[0]
overheat_count = df_transformer['temp_alert'].sum()

# Billing anomalies
billing_summary = df_billing.groupby('anomaly_type').size().reset_index(name='count')
total_billing_anomalies = billing_summary['count'].sum()
most_common_anomaly = billing_summary.loc[billing_summary['count'].idxmax(), 'anomaly_type']

In [13]:
# --- Step 2: Generate Combined Dashboard Image ---

sns.set(style="whitegrid")
fig, axs = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Power usage and anomalies
sns.barplot(data=daily_usage, x='substation_id', y='usage_kwh', hue='anomaly', ax=axs[0, 0])
axs[0, 0].set_title('Daily Power Usage per Substation')
axs[0, 0].tick_params(axis='x', rotation=45)

# Plot 2: Outage average duration per area
sns.barplot(data=outage_summary.reset_index(), x='area_id', y='avg_duration', ax=axs[0, 1])
axs[0, 1].set_title('Average Outage Duration per Area')
axs[0, 1].tick_params(axis='x', rotation=45)

# Plot 3: Transformer temperature alerts count
sns.countplot(data=df_transformer, x='temp_alert', ax=axs[1, 0])
axs[1, 0].set_title('Transformer Temperature Alerts (Temp > 80°C)')
axs[1, 0].set_xticklabels(['Normal', 'Critical'])

# Plot 4: Billing anomalies count by type
sns.barplot(data=billing_summary, x='anomaly_type', y='count', ax=axs[1, 1])
axs[1, 1].set_title('Billing Anomalies by Type')
axs[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
image_path = "combined_dashboard.png"
plt.savefig(image_path)
plt.close()



/tmp/ipykernel_578071/2676147142.py:19: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  axs[1, 0].set_xticklabels(['Normal', 'Critical'])


In [18]:
# --- Step 3: Generate HTML Email Body ---

today_str = datetime.now().strftime("%Y-%m-%d")

html_body = f"""
<html>
  <body>
    <h2>KBLC Daily Analytics Report — {today_str}</h2>

    <h3>1. Power Usage Summary</h3>
    <p>Total Substations: <b>{total_substations}</b></p>
    <p>Total Consumption: <b>{total_consumption:,.2f} kWh</b></p>
    <p>Anomalies Detected: <b>{anomaly_count}</b> (usage > 2 std deviations)</p>

    <h3>2. Outage Summary</h3>
    <p>Areas with Outages: <b>{total_areas}</b></p>
    <p>Average Outage Duration: <b>{avg_outage_duration:.2f} minutes</b></p>
    <p>Area with Highest Outages: <b>{max_outage_area}</b> ({max_outage_freq} outages)</p>

    <h3>3. Transformer Temperature Alerts</h3>
    <p>Total Transformers Monitored: <b>{total_transformers}</b></p>
    <p>Transformers Overheating (>80°C): <b>{overheat_count}</b></p>

    <h3>4. Billing Anomalies</h3>
    <p>Total Billing Anomalies: <b>{total_billing_anomalies}</b></p>
    <p>Most Common Anomaly: <b>{most_common_anomaly}</b></p>

    <h3>Attached: Combined dashboard visualization</h3>
    <img src="cid:dashboard_img" alt="Dashboard Image" style="max-width: 700px;"/>

    <p>Regards,<br>KBLC Analytics Team</p>
  </body>
</html>
"""

# --- Step 4: Send Email with HTML and Image Inline ---

import email.utils
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def send_kplc_report(sender, password, receiver, smtp_server='smtp.gmail.com', smtp_port=465):
    # Create multipart email
    msg = MIMEMultipart('related')
    msg['Subject'] = f"KBLC Daily Analytics Report — {today_str}"
    msg['From'] = sender
    msg['To'] = receiver
    msg['Date'] = email.utils.formatdate(localtime=True)

    # Alternative part for HTML
    msg_alternative = MIMEMultipart('alternative')
    msg.attach(msg_alternative)

    # Attach HTML
    msg_text = MIMEText(html_body, 'html')
    msg_alternative.attach(msg_text)

    # Attach image as inline content
    with open(image_path, 'rb') as img_file:
        img = MIMEImage(img_file.read())
        img.add_header('Content-ID', '<dashboard_img>')
        img.add_header('Content-Disposition', 'inline', filename=image_path)
        msg.attach(img)

    # Send email
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL(smtp_server, smtp_port, context=context) as server:
        server.login(sender, password)
        server.sendmail(sender, receiver, msg.as_string())

    print("✅ KPLC daily report email sent successfully.")

# --- Usage ---

# Replace these with your credentials & recipient
your_email = "petergatitu61@gmail.com"
your_password = "rota tvuh ackb khvx"
recipient_email = "petergatitu@gmail.com"

# Uncomment to send the email (make sure to set your credentials above)
send_kplc_report(your_email, your_password, recipient_email)


✅ KPLC daily report email sent successfully.
